In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#
#   Convert RGB images to CIE-LAB Color File

from __future__ import print_function
import argparse
import os

import numpy as np
import pickle
import random
from keras.preprocessing.image import list_pictures, array_to_img
from image_ext import list_pictures_in_multidir, load_imgs_asarray, img_dice_coeff

np.random.seed(2016)

Using TensorFlow backend.


In [2]:
def load_fnames(paths):
    f = open(paths)
    data1 = f.read()
    f.close()
    lines = data1.split('\n')
    #print(len(lines))
    # 最終行は空行なので消す
    del(lines[len(lines)-1])
    #print(len(lines))
    return lines

In [3]:
def make_fnames(fnames,fpath,fpath_mask,mask_ext):
    fnames_img = [];
    fnames_mask= [];
    
    for i in range(len(fnames)):
        fnames_img.append(fpath + '/' + fnames[i]);
        fnames_mask.append(fpath_mask + '/' + mask_ext + fnames[i]);
        
    return [fnames_img,fnames_mask]

In [4]:
def RGB2RGChroma(img):
    # convert RGB to RG Chromaticity Image
    # https://en.wikipedia.org/wiki/Rg_chromaticity
    ret = np.zeros((img.shape[0],img.shape[1],img.shape[2]),'float')
    
    R = img[:,:,0]
    G = img[:,:,1]
    B = img[:,:,2]
    I = R + G + B + 1
    r = (R+1)/I
    g = (G+1)/I
    b = (B+1)/I
    R2 = r*G / g
    B2 = ((1 - r - g)*G) / g
    
    ret[:,:,0] = r
    ret[:,:,1] = g
    ret[:,:,2] = 0
    
    return ret

In [5]:
def LabConversion(img):
    ret = cv2.cvtColor(img.astype('uint8'), cv2.COLOR_RGB2LAB)
    return ret

In [6]:
def LabAugumentation(img,L,a,b):
    ret = cv2.cvtColor(img.astype('uint8'), cv2.COLOR_BGR2LAB)
    # print(ret)
    ret[:,:,0] = ret[:,:,0] + L
    ret[:,:,1] = ret[:,:,1] + a
    ret[:,:,2] = ret[:,:,2] + b
    ret = np.clip(ret,0,255)
    ret = cv2.cvtColor(ret, cv2.COLOR_LAB2BGR)    
    return ret

In [13]:
#
#  MAIN STARTS FROM HERE
#
if __name__ == '__main__':
    
    target_size = (224, 224)
    dim_ordering = 'channels_last'
    
#    fnames = load_fnames('data.nnlab/list_files_all.txt')
#    [fpaths_xs_train,fpaths_ys_train] = make_fnames(fnames,'data.nnlab/image','data.nnlab/gt','')
#    [fpaths_xs_out,fpaths_ys_out] = make_fnames(fnames,'data.nnlab.LAB/img','data.nnlab/gt','')
    fnames = load_fnames('data/list_files_all.txt')
    [fpaths_xs_train,fpaths_ys_train] = make_fnames(fnames,'data/img','data/mask','OperatorA_')
    [fpaths_xs_out,fpaths_ys_out] = make_fnames(fnames,'data_augumented/img','data_augumented/mask','OperatorA_')

    if os.path.exists('data_augumented') == 0:
        os.mkdir('data_augumented')
        os.mkdir('data_augumented/img')
        os.mkdir('data_augumented/mask')
        
#    X_img = load_imgs_asarray(fpaths_xs_train, grayscale=False, target_size=target_size,
#                                dim_ordering=dim_ordering)
#    Y_img = load_imgs_asarray(fpaths_ys_train, grayscale=True, target_size=target_size,
#                                dim_ordering=dim_ordering)    

In [18]:
    print(X_img.shape)

(2249, 224, 224, 3)


True

In [22]:
    import matplotlib.pyplot as plt
    from PIL import Image 
    import cv2
    
    aug_div = [20,30,30]    
    
    fpath_img = 'data_augumented/img'
    fpath_mask = 'data_augumented/mask'
    
    for i in range(X_img.shape[0]):
        img = cv2.imread(fpaths_xs_train[i])
        mask = cv2.imread(fpaths_ys_train[i])
        
        fnames_img = fpath_img + '/' + fnames[i];
        cv2.imwrite(fnames_img, img)
        
        fnames_mask = fpath_mask + '/' + fnames[i];
        print(fnames_mask)
        cv2.imwrite(fnames_mask,mask)
        
        for j in range(10):
            aug_param = [ random.randint(-aug_div[0],aug_div[0]), random.randint(-aug_div[1],aug_div[1]),
                             random.randint(-aug_div[2],aug_div[2]) ]
            print(aug_param)
            im = LabAugumentation(img,aug_param[0],aug_param[1],aug_param[2])
            
            fnames_img = '%s/%03d_%s'%(fpath_img, j, fnames[i])
            fnames_mask = '%s/%03d_%s'%(fpath_mask, j, fnames[i])
            print(fnames_img)
            print(fnames_mask)
            cv2.imwrite(fnames_img, im)
            cv2.imwrite(fnames_mask, mask)
        

data_augumented/mask/C100_S1_I1.tiff
[-9, 29, 11]
data_augumented/img/000_C100_S1_I1.tiff
data_augumented/mask/000_C100_S1_I1.tiff
[-4, -4, -15]
data_augumented/img/001_C100_S1_I1.tiff
data_augumented/mask/001_C100_S1_I1.tiff
[-5, -23, 19]
data_augumented/img/002_C100_S1_I1.tiff
data_augumented/mask/002_C100_S1_I1.tiff
[-11, 11, -8]
data_augumented/img/003_C100_S1_I1.tiff
data_augumented/mask/003_C100_S1_I1.tiff
[1, 22, 2]
data_augumented/img/004_C100_S1_I1.tiff
data_augumented/mask/004_C100_S1_I1.tiff
[19, 8, 2]
data_augumented/img/005_C100_S1_I1.tiff
data_augumented/mask/005_C100_S1_I1.tiff
[15, 4, 3]
data_augumented/img/006_C100_S1_I1.tiff
data_augumented/mask/006_C100_S1_I1.tiff
[6, -23, 14]
data_augumented/img/007_C100_S1_I1.tiff
data_augumented/mask/007_C100_S1_I1.tiff
[-20, 24, -17]
data_augumented/img/008_C100_S1_I1.tiff
data_augumented/mask/008_C100_S1_I1.tiff
[-2, 14, 20]
data_augumented/img/009_C100_S1_I1.tiff
data_augumented/mask/009_C100_S1_I1.tiff
data_augumented/mask/C10

[2, 1, -16]
data_augumented/img/004_C100_S1_I3.tiff
data_augumented/mask/004_C100_S1_I3.tiff
[8, -6, 25]
data_augumented/img/005_C100_S1_I3.tiff
data_augumented/mask/005_C100_S1_I3.tiff
[-8, -24, -5]
data_augumented/img/006_C100_S1_I3.tiff
data_augumented/mask/006_C100_S1_I3.tiff
[-5, -21, 14]
data_augumented/img/007_C100_S1_I3.tiff
data_augumented/mask/007_C100_S1_I3.tiff
[1, -20, -16]
data_augumented/img/008_C100_S1_I3.tiff
data_augumented/mask/008_C100_S1_I3.tiff
[-17, 3, 17]
data_augumented/img/009_C100_S1_I3.tiff
data_augumented/mask/009_C100_S1_I3.tiff
data_augumented/mask/C100_S1_I4.tiff
[-20, -18, -3]
data_augumented/img/000_C100_S1_I4.tiff
data_augumented/mask/000_C100_S1_I4.tiff
[-9, -29, -4]
data_augumented/img/001_C100_S1_I4.tiff
data_augumented/mask/001_C100_S1_I4.tiff
[-1, -26, 8]
data_augumented/img/002_C100_S1_I4.tiff
data_augumented/mask/002_C100_S1_I4.tiff
[1, 29, 11]
data_augumented/img/003_C100_S1_I4.tiff
data_augumented/mask/003_C100_S1_I4.tiff
[-4, 11, -9]
data_au

data_augumented/mask/C100_S2_I11.tiff
[16, -4, -3]
data_augumented/img/000_C100_S2_I11.tiff
data_augumented/mask/000_C100_S2_I11.tiff
[1, -27, -18]
data_augumented/img/001_C100_S2_I11.tiff
data_augumented/mask/001_C100_S2_I11.tiff
[7, -7, 18]
data_augumented/img/002_C100_S2_I11.tiff
data_augumented/mask/002_C100_S2_I11.tiff
[-10, -29, -28]
data_augumented/img/003_C100_S2_I11.tiff
data_augumented/mask/003_C100_S2_I11.tiff
[-16, 24, 1]
data_augumented/img/004_C100_S2_I11.tiff
data_augumented/mask/004_C100_S2_I11.tiff
[-7, 30, 22]
data_augumented/img/005_C100_S2_I11.tiff
data_augumented/mask/005_C100_S2_I11.tiff
[-16, 25, 25]
data_augumented/img/006_C100_S2_I11.tiff
data_augumented/mask/006_C100_S2_I11.tiff
[-11, -30, 14]
data_augumented/img/007_C100_S2_I11.tiff
data_augumented/mask/007_C100_S2_I11.tiff
[7, -18, -30]
data_augumented/img/008_C100_S2_I11.tiff
data_augumented/mask/008_C100_S2_I11.tiff
[-18, 30, 20]
data_augumented/img/009_C100_S2_I11.tiff
data_augumented/mask/009_C100_S2_I11

[-17, -1, 30]
data_augumented/img/008_C100_S2_I5.tiff
data_augumented/mask/008_C100_S2_I5.tiff
[18, -23, -23]
data_augumented/img/009_C100_S2_I5.tiff
data_augumented/mask/009_C100_S2_I5.tiff
data_augumented/mask/C100_S2_I6.tiff
[-6, -16, 23]
data_augumented/img/000_C100_S2_I6.tiff
data_augumented/mask/000_C100_S2_I6.tiff
[1, 8, 27]
data_augumented/img/001_C100_S2_I6.tiff
data_augumented/mask/001_C100_S2_I6.tiff
[7, -2, -22]
data_augumented/img/002_C100_S2_I6.tiff
data_augumented/mask/002_C100_S2_I6.tiff
[-11, 1, -27]
data_augumented/img/003_C100_S2_I6.tiff
data_augumented/mask/003_C100_S2_I6.tiff
[-15, -17, 30]
data_augumented/img/004_C100_S2_I6.tiff
data_augumented/mask/004_C100_S2_I6.tiff
[3, -23, -2]
data_augumented/img/005_C100_S2_I6.tiff
data_augumented/mask/005_C100_S2_I6.tiff
[-2, -8, -19]
data_augumented/img/006_C100_S2_I6.tiff
data_augumented/mask/006_C100_S2_I6.tiff
[7, -1, -2]
data_augumented/img/007_C100_S2_I6.tiff
data_augumented/mask/007_C100_S2_I6.tiff
[14, -17, 18]
data

[19, -28, 20]
data_augumented/img/006_C101_S1_I13.tiff
data_augumented/mask/006_C101_S1_I13.tiff
[18, -26, -9]
data_augumented/img/007_C101_S1_I13.tiff
data_augumented/mask/007_C101_S1_I13.tiff
[19, -4, -25]
data_augumented/img/008_C101_S1_I13.tiff
data_augumented/mask/008_C101_S1_I13.tiff
[-16, 18, 8]
data_augumented/img/009_C101_S1_I13.tiff
data_augumented/mask/009_C101_S1_I13.tiff
data_augumented/mask/C101_S1_I14.tiff
[-16, 0, -6]
data_augumented/img/000_C101_S1_I14.tiff
data_augumented/mask/000_C101_S1_I14.tiff
[17, 20, -23]
data_augumented/img/001_C101_S1_I14.tiff
data_augumented/mask/001_C101_S1_I14.tiff
[16, 9, 30]
data_augumented/img/002_C101_S1_I14.tiff
data_augumented/mask/002_C101_S1_I14.tiff
[-2, -27, 0]
data_augumented/img/003_C101_S1_I14.tiff
data_augumented/mask/003_C101_S1_I14.tiff
[20, -6, -10]
data_augumented/img/004_C101_S1_I14.tiff
data_augumented/mask/004_C101_S1_I14.tiff
[20, 5, -12]
data_augumented/img/005_C101_S1_I14.tiff
data_augumented/mask/005_C101_S1_I14.tif

...

[-19, 25, 25]
data_augumented/img/005_C99_S2_I14.tiff
data_augumented/mask/005_C99_S2_I14.tiff
[-3, 22, 21]
data_augumented/img/006_C99_S2_I14.tiff
data_augumented/mask/006_C99_S2_I14.tiff
[16, 12, -4]
data_augumented/img/007_C99_S2_I14.tiff
data_augumented/mask/007_C99_S2_I14.tiff
[20, 21, -10]
data_augumented/img/008_C99_S2_I14.tiff
data_augumented/mask/008_C99_S2_I14.tiff
[17, 5, -21]
data_augumented/img/009_C99_S2_I14.tiff
data_augumented/mask/009_C99_S2_I14.tiff
data_augumented/mask/C99_S2_I15.tiff
[0, 9, -15]
data_augumented/img/000_C99_S2_I15.tiff
data_augumented/mask/000_C99_S2_I15.tiff
[-9, -30, -22]
data_augumented/img/001_C99_S2_I15.tiff
data_augumented/mask/001_C99_S2_I15.tiff
[14, 3, 8]
data_augumented/img/002_C99_S2_I15.tiff
data_augumented/mask/002_C99_S2_I15.tiff
[-19, -3, -21]
data_augumented/img/003_C99_S2_I15.tiff
data_augumented/mask/003_C99_S2_I15.tiff
[7, 4, 2]
data_augumented/img/004_C99_S2_I15.tiff
data_augumented/mask/004_C99_S2_I15.tiff
[-5, 9, -8]
data_augume

[-9, 16, 26]
data_augumented/img/006_C99_S2_I9.tiff
data_augumented/mask/006_C99_S2_I9.tiff
[20, -29, -24]
data_augumented/img/007_C99_S2_I9.tiff
data_augumented/mask/007_C99_S2_I9.tiff
[-17, -15, 20]
data_augumented/img/008_C99_S2_I9.tiff
data_augumented/mask/008_C99_S2_I9.tiff
[8, 11, 2]
data_augumented/img/009_C99_S2_I9.tiff
data_augumented/mask/009_C99_S2_I9.tiff
data_augumented/mask/C9_S1_I1.tiff
[4, -24, -1]
data_augumented/img/000_C9_S1_I1.tiff
data_augumented/mask/000_C9_S1_I1.tiff
[-20, 20, 3]
data_augumented/img/001_C9_S1_I1.tiff
data_augumented/mask/001_C9_S1_I1.tiff
[-13, -25, 10]
data_augumented/img/002_C9_S1_I1.tiff
data_augumented/mask/002_C9_S1_I1.tiff
[11, 7, -12]
data_augumented/img/003_C9_S1_I1.tiff
data_augumented/mask/003_C9_S1_I1.tiff
[10, -30, 9]
data_augumented/img/004_C9_S1_I1.tiff
data_augumented/mask/004_C9_S1_I1.tiff
[-5, -29, 26]
data_augumented/img/005_C9_S1_I1.tiff
data_augumented/mask/005_C9_S1_I1.tiff
[-12, 6, -26]
data_augumented/img/006_C9_S1_I1.tiff


data_augumented/mask/C9_S1_I4.tiff
[13, -9, 6]
data_augumented/img/000_C9_S1_I4.tiff
data_augumented/mask/000_C9_S1_I4.tiff
[-6, -27, 6]
data_augumented/img/001_C9_S1_I4.tiff
data_augumented/mask/001_C9_S1_I4.tiff
[-13, -26, -25]
data_augumented/img/002_C9_S1_I4.tiff
data_augumented/mask/002_C9_S1_I4.tiff
[-6, -23, -9]
data_augumented/img/003_C9_S1_I4.tiff
data_augumented/mask/003_C9_S1_I4.tiff
[4, 10, -2]
data_augumented/img/004_C9_S1_I4.tiff
data_augumented/mask/004_C9_S1_I4.tiff
[18, 8, -25]
data_augumented/img/005_C9_S1_I4.tiff
data_augumented/mask/005_C9_S1_I4.tiff
[-3, -27, 22]
data_augumented/img/006_C9_S1_I4.tiff
data_augumented/mask/006_C9_S1_I4.tiff
[14, 12, -5]
data_augumented/img/007_C9_S1_I4.tiff
data_augumented/mask/007_C9_S1_I4.tiff
[5, 13, 27]
data_augumented/img/008_C9_S1_I4.tiff
data_augumented/mask/008_C9_S1_I4.tiff
[11, 1, -12]
data_augumented/img/009_C9_S1_I4.tiff
data_augumented/mask/009_C9_S1_I4.tiff
data_augumented/mask/C9_S1_I5.tiff
[4, 1, -12]
data_augumented/

[-17, 16, -25]
data_augumented/img/001_C9_S2_I12.tiff
data_augumented/mask/001_C9_S2_I12.tiff
[-7, -4, -12]
data_augumented/img/002_C9_S2_I12.tiff
data_augumented/mask/002_C9_S2_I12.tiff
[9, 11, 17]
data_augumented/img/003_C9_S2_I12.tiff
data_augumented/mask/003_C9_S2_I12.tiff
[-15, -28, 3]
data_augumented/img/004_C9_S2_I12.tiff
data_augumented/mask/004_C9_S2_I12.tiff
[13, 22, 23]
data_augumented/img/005_C9_S2_I12.tiff
data_augumented/mask/005_C9_S2_I12.tiff
[14, -18, -21]
data_augumented/img/006_C9_S2_I12.tiff
data_augumented/mask/006_C9_S2_I12.tiff
[18, 13, -24]
data_augumented/img/007_C9_S2_I12.tiff
data_augumented/mask/007_C9_S2_I12.tiff
[-6, 1, 12]
data_augumented/img/008_C9_S2_I12.tiff
data_augumented/mask/008_C9_S2_I12.tiff
[-17, 13, 5]
data_augumented/img/009_C9_S2_I12.tiff
data_augumented/mask/009_C9_S2_I12.tiff
data_augumented/mask/C9_S2_I13.tiff
[-5, -11, -21]
data_augumented/img/000_C9_S2_I13.tiff
data_augumented/mask/000_C9_S2_I13.tiff
[-16, -17, 27]
data_augumented/img/00

[-6, 13, 19]
data_augumented/img/009_C9_S2_I6.tiff
data_augumented/mask/009_C9_S2_I6.tiff
data_augumented/mask/C9_S2_I7.tiff
[0, 29, 14]
data_augumented/img/000_C9_S2_I7.tiff
data_augumented/mask/000_C9_S2_I7.tiff
[-18, -1, 2]
data_augumented/img/001_C9_S2_I7.tiff
data_augumented/mask/001_C9_S2_I7.tiff
[3, 8, 5]
data_augumented/img/002_C9_S2_I7.tiff
data_augumented/mask/002_C9_S2_I7.tiff
[9, 21, 25]
data_augumented/img/003_C9_S2_I7.tiff
data_augumented/mask/003_C9_S2_I7.tiff
[-7, -14, 28]
data_augumented/img/004_C9_S2_I7.tiff
data_augumented/mask/004_C9_S2_I7.tiff
[19, 10, 16]
data_augumented/img/005_C9_S2_I7.tiff
data_augumented/mask/005_C9_S2_I7.tiff
[7, -27, 28]
data_augumented/img/006_C9_S2_I7.tiff
data_augumented/mask/006_C9_S2_I7.tiff
[9, 22, -24]
data_augumented/img/007_C9_S2_I7.tiff
data_augumented/mask/007_C9_S2_I7.tiff
[2, -17, 2]
data_augumented/img/008_C9_S2_I7.tiff
data_augumented/mask/008_C9_S2_I7.tiff
[-8, -30, -22]
data_augumented/img/009_C9_S2_I7.tiff
data_augumented/m